### Dataset Description
The dataset contains bird images, divided into train and test splits. The images are inside test_images and train_images folders.

The labels of the training images are inside train_images.csv file. In this file, the first column is image_path and the second one is the label (1 - 200). The test_images_samples.csv includes a row id with a dummy label. The final goal of the challenge is to change the label column to the predicted label.

The class_names.npy is a dictionary including the name of each label. Load the file using the following code: np.load("class_names.npy", allow_pickle=True).item()

The structure of the final submission should be exactly the same as the test_images_samples.csv! Otherwise, it will fail.

Files

- train_images - the training images
- test_images - the test images
- test_images_sample.csv - a sample submission file in the correct format
- test_images_path.csv - path to test file images
- train_images.csv - supplemental information about the data
- class_names.npy - this file includes the name of each label
- attributes.npy - this file includes the attributes which are extra information for each class.
- attributes.txt - this file includes the attribute names which are extra information for each class.

### Ideas and links: 

1. Train from scratch 
1. Use pretrained cnn (resnet18) and finetune
1. Use pretrained and do feature extraction

**complete tutorials**
- [Link](https://docs.pytorch.org/tutorials/beginner/transfer_learning_tutorial.html) to tutorial on pytorch
- [link](https://www.youtube.com/watch?v=t6oHGXt04ik) to tutorial on transfer learning with a resnet on youtube

**Creating a custom dataset**
- [link](https://docs.pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files) To pytorch tutorial of creating a custom dataset for your files


In [1]:
# Setup and imports
import numpy as np
import pandas as pd
import numpy as np
import os
from PIL import Image
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


**Exploring class features**

In [2]:
class_names = np.load("class_names.npy", allow_pickle=True).item()
print(F"First class: {class_names}") 
attributes = np.load("attributes.npy", allow_pickle=True)
print(F"Attributes shape: {attributes.shape}") 


First class: {np.str_('001.Black_footed_Albatross'): 1, np.str_('002.Laysan_Albatross'): 2, np.str_('003.Sooty_Albatross'): 3, np.str_('004.Groove_billed_Ani'): 4, np.str_('005.Crested_Auklet'): 5, np.str_('006.Least_Auklet'): 6, np.str_('007.Parakeet_Auklet'): 7, np.str_('008.Rhinoceros_Auklet'): 8, np.str_('009.Brewer_Blackbird'): 9, np.str_('010.Red_winged_Blackbird'): 10, np.str_('011.Rusty_Blackbird'): 11, np.str_('012.Yellow_headed_Blackbird'): 12, np.str_('013.Bobolink'): 13, np.str_('014.Indigo_Bunting'): 14, np.str_('015.Lazuli_Bunting'): 15, np.str_('016.Painted_Bunting'): 16, np.str_('017.Cardinal'): 17, np.str_('018.Spotted_Catbird'): 18, np.str_('019.Gray_Catbird'): 19, np.str_('020.Yellow_breasted_Chat'): 20, np.str_('021.Eastern_Towhee'): 21, np.str_('022.Chuck_will_Widow'): 22, np.str_('023.Brandt_Cormorant'): 23, np.str_('024.Red_faced_Cormorant'): 24, np.str_('025.Pelagic_Cormorant'): 25, np.str_('026.Bronzed_Cowbird'): 26, np.str_('027.Shiny_Cowbird'): 27, np.str_('0

### Setting up dataset and dataloaders
creating train and validation set (80:20)

In [ ]:
from torchvision.io import read_image
from torchvision import transforms
from torch.utils.data import DataLoader, random_split

torch.manual_seed(42)

class BirdDataset():
    def __init__(self, annotations_file, img_dir, img_col_index=0, label_col_index=1, transform=None, target_transform=None):
        """
        image_col_index: troubleshooting different csv formats( 0 for train , 1 for test)
        label_col_index: (1 for train, 2 for test)

        """

        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.img_col_index = img_col_index     
        self.label_col_index = label_col_index 
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        raw_val = self.img_labels.iloc[idx, self.img_col_index]
        
        
        filename = str(raw_val)
        filename = os.path.basename(filename)  
        img_path = os.path.join(self.img_dir, filename)
        image = read_image(img_path)

            
    
        label = self.img_labels.iloc[idx, self.label_col_index]

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
            
        return image, label

standard_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

In [8]:
full_dataset = BirdDataset(
    annotations_file='train_images.csv',
    img_dir='train_images/train_images',
    transform=standard_transform,
    img_col_index=0,   # Standaard
    label_col_index=1  # Standaard
)

#Splitting the dataset into train and val 80/20
train_size = int(0.8*len(full_dataset))
val_size = int(len(full_dataset)) - train_size

train_set, val_set = random_split(full_dataset, [train_size, val_size])

#Setting up Dataloaders
train_loader = DataLoader(train_set, batch_size = 32, shuffle = True )
val_loader = DataLoader(val_set, batch_size = 32, shuffle = False)

images, labels = next(iter(train_loader))
print(f"Batch size: {images.shape}")


Batch size: torch.Size([32, 3, 224, 224])


**Setting up test set**

In [9]:
test_dataset = BirdDataset(
    annotations_file='test_images_path.csv', 
    img_dir='test_images/test_images',
    transform=standard_transform,
    img_col_index=1,  
    label_col_index=2
)

test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = False)
print(f'length testset = {len(test_dataset)}')

test_images, _ = next(iter(test_loader))
print(f"Test batch shape: {test_images.shape}")

length testset = 4000
Test batch shape: torch.Size([32, 3, 224, 224])
